**pdf2image**

installation





In [ ]:
!pip install pdf2image
!apt-get install poppler-utils

conversion

In [2]:
from pdf2image import convert_from_path

In [3]:
images = convert_from_path('SAO KÊ 2.pdf')

In [4]:
!mkdir pages

In [5]:
for i in range(len(images)):
  images[i].save('pages/page'+ str(i) +'.jpg', 'JPEG')

In [ ]:
!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf>=3.20.3,<5.0.0 --no-cache-dir
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

In [ ]:
import cv2
import layoutparser as lp
image = cv2.imread("/content/pages/page1.jpg")

image = image[..., ::-1]

# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

In [ ]:
layout

In [ ]:
x_1=0
y_1=0
x_2=0
y_2=0
for l in layout:
  if l.type == 'Table':
    x_1 = int(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)
    print(l.block)

In [11]:
im = cv2.imread("/content/pages/page1.jpg")

In [ ]:
cv2.imwrite('extracted_image.jpg', im[y_1:y_2, x_1:x_2])

In [13]:
from paddleocr import PaddleOCR, draw_ocr

In [ ]:
orc = PaddleOCR(lang = 'en')

In [ ]:
ocr = PaddleOCR(lang='en')
image_path = '/content/extracted_image.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)[0]

In [ ]:
print(output)

In [17]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [18]:
image_boxes = image_cv.copy()

In [19]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [20]:
cv2.imwrite('detections.jpg', image_boxes)

True

Reconstruction

In [21]:
im = image_cv.copy()

In [22]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [23]:
cv2.imwrite('horiz_vert.jpg',im)

True

Non-max suppression

In [24]:
import tensorflow as tf
import numpy as np

In [25]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [26]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[  3   5  11  16  18  20  25  27  32  35  40  41  46  48  49  50  51  56
  58  63  64  65  70  72  77  80  85  87  92  94  99 101 106 109 113 115
 119 121 126 128 133 137 141 142 143 144 149 153]


In [27]:
im_nms = image_cv.copy()

In [28]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [29]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [31]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [ ]:
print(vert_out)

In [33]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[  7   8  20  75  92  94 109 153]


In [34]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

In [30]:
cv2.imwrite('im_nms.jpg',im_nms)

True

Convert to CSV

In [ ]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

In [ ]:
unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

In [ ]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

In [38]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [39]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [40]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [41]:
out_array=np.array(out_array)

In [ ]:
out_array

In [47]:
import pandas as pd

In [48]:
pd.DataFrame(out_array).to_csv('sample.csv')